In [ ]:
import pandas as pd
import numpy as np
import influxdb_client
import matplotlib.pyplot as plt

from influxdb_client.client.write_api import SYNCHRONOUS

org = "fh_soest"
url = "https://eu-central-1-1.aws.cloud2.influxdata.com"
token = "Bt4r9yYLtH7gitVgnwgULJ47UFmymn18378SsnJMdRXFVz_S14WmPI5JYU-CV4MtpzThGFVhEUJ6htpLX0M43w=="
bucket = "fh_bucket"
write_client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)

write_api = write_client.write_api(write_options=SYNCHRONOUS)
query_api = write_client.query_api()

In [ ]:
# SIMPLE QUERY
query1 = """from(bucket: "fh_bucket")
            |> range(start: 2023-12-18T10:00:00Z, stop: now())
            |> filter(fn: (r) => r["_measurement"] == "Experiment")
            |> filter(fn: (r) => r["Device_ID"] == "0013A20041BDFD8D")
            |> filter(fn: (r) => r["Location"] == "Soest")
            |> filter(fn: (r) => r["_field"] == "Light")
            |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
         """

query2 = """from(bucket: "fh_bucket")
            |> range(start: 2023-12-18T10:00:00Z, stop: now())
            |> filter(fn: (r) => r["_measurement"] == "Experiment")
            |> filter(fn: (r) => r["Device_ID"] == "0013A20041BDFD8D")
            |> filter(fn: (r) => r["Location"] == "Soest")
            |> filter(fn: (r) => r["_field"] == "Temperature")
            |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
         """

query3 = """from(bucket: "fh_bucket")
            |> range(start: 2023-12-18T10:00:00Z, stop: now())
            |> filter(fn: (r) => r["_measurement"] == "Experiment")
            |> filter(fn: (r) => r["Device_ID"] == "0013A20041BDFD8D")
            |> filter(fn: (r) => r["Location"] == "Soest")
            |> filter(fn: (r) => r["_field"] == "Altitude")
            |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
         """

query4 = """from(bucket: "fh_bucket")
            |> range(start: 2023-12-18T10:00:00Z, stop: now())
            |> filter(fn: (r) => r["_measurement"] == "Experiment")
            |> filter(fn: (r) => r["Device_ID"] == "0013A20041BDFD8D")
            |> filter(fn: (r) => r["Location"] == "Soest")
            |> filter(fn: (r) => r["_field"] == "Pressure")
            |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
         """

query5 = """from(bucket: "fh_bucket")
            |> range(start: 2023-12-18T10:00:00Z, stop: now())
            |> filter(fn: (r) => r["_measurement"] == "Experiment")
            |> filter(fn: (r) => r["Device_ID"] == "0013A20041BDFD8D")
            |> filter(fn: (r) => r["Location"] == "Soest")
            |> filter(fn: (r) => r["_field"] == "Battery")
            |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
         """

In [ ]:
# Query: using Pandas DataFrame
df1 = query_api.query_data_frame(query1)
df2 = query_api.query_data_frame(query2)
df3 = query_api.query_data_frame(query3)
df4 = query_api.query_data_frame(query4)
df5 = query_api.query_data_frame(query5)

In [ ]:
df1 = df1[["_time", "Device_ID", "Location", "_measurement", "Light"]]
df2 = df2[["_time", "Device_ID", "Location", "_measurement", "Temperature"]]
df3 = df3[["_time", "Device_ID", "Location", "_measurement", "Altitude"]]
df4 = df4[["_time", "Device_ID", "Location", "_measurement", "Pressure"]]
df5 = df5[["_time", "Device_ID", "Location", "_measurement", "Battery"]]

In [ ]:
df5.head()

In [ ]:
# Merge dataframes on common columns
dfs = [df1, df2, df3, df4, df5]

# Starting with the first dataframe
merged_df = dfs[0]

# Iterating over the rest of the dataframes and merging
for df in dfs[1:]:
    merged_df = pd.merge(merged_df, df, on=['_time', 'Device_ID', 'Location', '_measurement'], how='inner')

# Rename the columns
result_df = merged_df.rename(columns={'_Time': 'Time', '_measurement': 'Measurement'})

In [ ]:
result_df.head()

In [ ]:
# Convert the 'time' column to datetime format
result_df['_time'] = pd.to_datetime(result_df['_time'])

# Filter rows where 'time' is on or after 29 January 2024
filtered_df = result_df[result_df['_time'] >= '2024-01-29']

# Convert datetime values to timezone-naive format
filtered_df['_time'] = filtered_df['_time'].dt.tz_localize(None)

In [ ]:
# Plot the time series data
plt.figure(figsize=(10, 6))
plt.plot(filtered_df['_time'], filtered_df['Light'], label='Time Series Data', color='blue')

plt.grid(True)
plt.xlabel('Time')
plt.ylabel('Light Value')
plt.title('Time Series Plot')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
filtered_df

In [ ]:
# Save the filtered DataFrame to an Excel file
filtered_df.to_excel('filtered_data.xlsx', index=True)